Installing necessary prereqs: [NetworkX](https://networkx.org/documentation/stable/reference/index.html) for graphing, [PuLP](https://coin-or.github.io/pulp/main/index.html) for linear programming.

In [11]:
%pip install networkx pulp

Note: you may need to restart the kernel to use updated packages.


In [12]:
import json
import networkx as nx

fileName = "map_ia_nw_weights"

file = open("map_data/" + fileName + ".json")
data = json.load(file)

# adjacencies = data["adj"]
# # TODO: Shits the bed if there's anything other than letters and _
# countyNames = data["GEOID"]
# pop = data["pop"]

adjacencies = []
countyNames = []
pop = []
weights = []
for countyInfo in data:
    countyNames.append(countyInfo["GEOID"])
    adjacencies.append(countyInfo["adj"])
    pop.append(countyInfo["pop"])
    weights.append(countyInfo["weights"])


# Number of districts
NUM_DISTRICTS = 4

# Population Tolerance
ALPHA = .05

In [13]:
from random import Random

stateGraph = nx.Graph()

r =  Random(51535162)
totalPop = 0
# Add the nodes as necessary

for i in range(len(adjacencies)):
    # Adds node for each set of adjacencies
    totalPop += pop[i]
    # Adds necessary edges; double-adding doesn't matter
    # try:
    for j in range(len(adjacencies[i])):
        # print(i,j)
        stateGraph.add_edge(countyNames[i], countyNames[adjacencies[i][j]], weight=float(weights[i][j]))
    # except:
    #         print(i)
    #         stateGraph.add_edge(countyNames[i], countyNames[adjacencies[i]], weight=weights[i])

# Population per district
targetPop = totalPop / NUM_DISTRICTS

# nx.draw(stateGraph)

In [14]:

import pulp 

import csv


def CutPartition(remainingGraph):
    prog = pulp.LpProblem("CutPartition", pulp.LpMinimize)

    # Defining Partition Binary
    xVars = []
    xVarMap = {}
    for n in remainingGraph.nodes:
        var = pulp.LpVariable("x" + str(n), cat=pulp.LpBinary)
        xVarMap[n] = var
        xVars.append(var)
        prog += var
    # Defining crossing binary
    edgeVars = []
    weights = []
    for n, m in remainingGraph.edges:
        edgeVal = pulp.LpAffineExpression([(xVarMap[n], 1), (xVarMap[m], -1)])
        absEdgeVal = pulp.LpVariable('y' + str(n) + str(m), cat=pulp.LpBinary)

        # Tries to fix absolute value prob
        prog += absEdgeVal >= edgeVal
        prog += absEdgeVal >= -edgeVal

        edgeVars.append(absEdgeVal)
        weights.append(remainingGraph[n][m]['weight'])
    # Subset sum
    popVariables = []
    for i in range(len(xVars)):
        popVariables.append(pulp.LpAffineExpression(xVars[i] * pop[i]))
    # Max Pop
    prog += pulp.lpSum(popVariables) <= targetPop * (1 + ALPHA)
    # Min Pop
    prog += pulp.lpSum(popVariables) >= targetPop * (1 - ALPHA)

    # Source Partition Connected
    for i in range(len(xVars)):
        sumParts = []
        for j in range(len(xVars)):
            if i == j: break
            if remainingGraph.has_edge(i, j):
                sumParts.append(remainingGraph[i][j]['weight'] * xVars[j])
        totalLH = pulp.LpAffineExpression(xVars[i] * pulp.lpSum(sumParts))
        prog += pulp.LpConstraint(totalLH, pulp.LpConstraintGE, 0)
    # Define Minimization Target
    # Edgeweights
    edgeWeightVars = []
    for i in range(len(edgeVars)):
        edgeWeightVars.append(pulp.LpAffineExpression(edgeVars[i] * weights[i]))
    prog += pulp.LpAffineExpression(pulp.lpSum(edgeWeightVars), name="Z")

    print("Sending to solver...")
    Solver_name = 'PULP_CBC_CMD'
    solver = pulp.getSolver(Solver_name, threads=12)
    soln = prog.solve(solver)

    print(soln)

    toReturn = {}

    districtPop = 0
    for i in range(len(xVars)):
        if(xVars[i].value() == 1):
            districtPop += pop[i]
            toReturn[xVars[i].name] = xVars[i].value()
            print(xVars[i].name, xVars[i].value())
    if(districtPop >= targetPop * (1 + ALPHA) or districtPop <= targetPop * (1-ALPHA)):
        raise ValueError("District Generated not of Proper Population") 
    return toReturn


remainingGraph = stateGraph.copy()
districts = []
while len(districts) < NUM_DISTRICTS - 1:
    cutResult = CutPartition(remainingGraph)
    newDistrict = []
    for d in cutResult.keys():
        # Gets rid of the x
        # print(d)
        newDistrict.append(d[1:])
    # Appends the new district to our list of districts
    districts.append(newDistrict)
    oldSize = remainingGraph.order()
    # Removes those nodes from the district
    remainingGraph.remove_nodes_from(newDistrict)
    newSize = remainingGraph.order()
    # Verify that all of the nodes were removed
    if(newSize != oldSize - len(newDistrict)):
        print(remainingGraph.nodes)
        print(newDistrict)
        raise(ValueError("Districts were not removed appropriately"))
    # print(newDistrict)
newDistrict = []
for d in remainingGraph.nodes:
    # Gets rid of the x
    newDistrict.append(d)
districts.append(newDistrict)

#Confirm total size is appropriate
totalCountiesinDistricts = 0
for d in districts:
    totalCountiesinDistricts += len(d)
if(totalCountiesinDistricts != len(countyNames)):
    raise(ValueError("Number of districts in plan does not match number of districts."))


file = open("district_outputs/" + fileName + '.csv', 'w')
w = csv.writer(file)
w.writerow(["district", "precinct"])
for i in range(len(districts)):
    for j in range(len(districts[i])):
        w.writerow([i + 1, districts[i][j]])
file.close()

Sending to solver...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/z3/3qb7cv41693fmz3b6hbntjkr0000gn/T/84d937a3f70843f1ae1b3440d58bf358-pulp.mps -threads 12 -timeMode elapsed -branch -printingOptions all -solution /var/folders/z3/3qb7cv41693fmz3b6hbntjkr0000gn/T/84d937a3f70843f1ae1b3440d58bf358-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 550 COLUMNS
At line 2945 RHS
At line 3491 BOUNDS
At line 3813 ENDATA
Problem MODEL has 545 rows, 321 columns and 1530 elements
Coin0008I MODEL read with 0 errors
threads was changed from 0 to 12
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 439 rows, 320 columns (320 integer (320 of which binary)) and 1406 elements
Cbc0038I Initial state - 104 integers unsatisfied sum - 16.